In [ ]:
#Import Statements
import sys
sys.path.insert(0, r'C:\Users\Matthew Arrieta\Desktop\Project3Testing\TextInput')
sys.path.insert(0, r'C:\Users\Matthew Arrieta\Desktop\Project3Testing\TestFiles')
sys.path.insert(0, r'C:\Users\Matthew Arrieta\Desktop\Project3Testing\Vectorizations')
sys.path.insert(0, r'C:\Users\Matthew Arrieta\Desktop\Project3Testing\Clustering')
from ClusterHDBSCAN import clusterHDBSCANNoLabels, clusterHDBSCANLabels, groupLabels


import os, itertools
import warnings
from matplotlib.pyplot import text
from numpy import vectorize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
import re
import numpy as np
import warnings
import statistics
import math
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
#from JaccardIndexUpdated import JaccardIndex, ClustersData, summary
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import completeness_score
#from VectorizeHelperFunction import allScores
from CleanText import getCleanText, getText
from vectorize import vectorizeTFIDF, vectorizeTFIDFUMAP, vectorizeTFIDFSVD
import umap
import umap.plot
import datashader as ds
import datashader.transfer_functions as tf
import datashader.bundling as bd
import matplotlib.pyplot as plt
import colorcet
import matplotlib.colors
import matplotlib.cm
import bokeh.plotting as bpl
import bokeh.transform as btr
from django.urls import re_path as url
import holoviews as hv
import holoviews.operation.datashader as hd 
import umap
import umap.plot
import hdbscan
import time
import seaborn as sns
from sklearn.metrics.cluster import normalized_mutual_info_score
from gensim.models import LdaModel, HdpModel
from gensim import corpora
from LDAKeywordExtraction import keywordsResultsTestingLDA, keywordResultsClusterLDA



In [ ]:
file_path_l = []
for root, dirs, files in os.walk(r"C:\Users\Matthew Arrieta\Desktop\Project3Testing\TestFiles\KeywordFilesTight"):
    for filename in files:
        print(root)
        file_path_l.append(os.path.join(root, filename))

text_l = []
for file_path in file_path_l:
    filey = open(file_path,encoding="mbcs")
    text_l.append(filey.read())
    filey.close()

outlier_num = 0
types_l = []
for namey in file_path_l:
    splitted = namey.split('\\')
    print(splitted[7])
    if splitted[7] == 'Outliers':
        # text_labels.append(file_path.split('\\')[-2])
        types_l.append(f"Outlier-{outlier_num}")
        outlier_num += 1
    else:
        types_l.append(splitted[7])
print(types_l)

In [3]:
text_l, labels = getCleanText(r"C:\Users\Matthew Arrieta\Desktop\Project3Testing\TestFiles\KeywordFilesTight", RemoveNums = True, lem=True, extendStopWords= False)
#print(labels)

In [ ]:
#Import Raw Text
rawText, rawlabels = getText("KeywordFilesTight")
#print(rawText[0])

In [4]:
vectorized = vectorizeTFIDFUMAP(text_l, labels)

In [11]:
df = clusterHDBSCANNoLabels(vectorized, printSummary= False)
print(df.columns)
print(list(df['Cluster Label']))

clusterLabels = df['Cluster Label']

Index(['Cluster Label', 'Cluster probabilities', 'Outlier Scores'], dtype='object')
[7, 7, 7, 7, 7, 7, 7, 17, 17, 17, 17, 17, 17, 17, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 12, 13, 13, 12, 13, 13, 13, -1, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 14, 14, 14, 15, 15, 15, 15, 15, 11, 11, 11, 11, 11, 11, 8, 8, 8, 8, 8, 4, 8, 3, 4, 4, 4, 4, 4, 4, -2, -3, 16, 16, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [5]:
def getKeywordsYake(clusterLabels):
  clusterLabelsNumpy = np.array(clusterLabels)
  seen = []
  groups = [""] * len(set(clusterLabels))
  count = 0 
  for x in range(len(clusterLabels)):
    if(clusterLabels[x] not in seen):
      seen.append(clusterLabels[x])
      locations = np.where(clusterLabelsNumpy == clusterLabels[x])[0].tolist()
      for x in (locations):
        groups[count] = groups[count] + " " + text_l[x]
      count = count + 1

  keywords = []
  for x in groups:
    currentKeyword = kw_extractor.extract_keywords(x)
    keywords.append(currentKeyword)
  return keywords


In [ ]:
print(keywordResultsClusterLDA(groupLabels(clusterLabels, text_l)))

In [ ]:
#Using LDA and HDP
def order_subset_by_coherence(dirichlet_model, bow_corpus, num_topics=10, num_keywords=10):
    """
    Orders topics based on their average coherence across the corpus

    Parameters
    ----------
        dirichlet_model : gensim.models.type_of_model
        bow_corpus : list of lists (contains (id, freq) tuples)
        num_topics : int (default=10)
        num_keywords : int (default=10)

    Returns
    -------
        ordered_topics, ordered_topic_averages: list of lists and list
    """
    if type(dirichlet_model) == LdaModel:
        shown_topics = dirichlet_model.show_topics(num_topics=num_topics, 
                                                   num_words=num_keywords,
                                                   formatted=False)
    elif type(dirichlet_model)  == HdpModel:
        shown_topics = dirichlet_model.show_topics(num_topics=150, # return all topics
                                                   num_words=num_keywords,
                                                   formatted=False)
    model_topics = [[word[0] for word in topic[1]] for topic in shown_topics]
    topic_corpus = dirichlet_model.__getitem__(bow=bow_corpus, eps=0) # cutoff probability to 0 

    topics_per_response = [response for response in topic_corpus]
    flat_topic_coherences = [item for sublist in topics_per_response for item in sublist]

    significant_topics = list(set([t_c[0] for t_c in flat_topic_coherences])) # those that appear
    topic_averages = [sum([t_c[1] for t_c in flat_topic_coherences if t_c[0] == topic_num]) / len(bow_corpus) \
                      for topic_num in significant_topics]

    topic_indexes_by_avg_coherence = [tup[0] for tup in sorted(enumerate(topic_averages), key=lambda i:i[1])[::-1]]

    significant_topics_by_avg_coherence = [significant_topics[i] for i in topic_indexes_by_avg_coherence]
    ordered_topics = [model_topics[i] for i in significant_topics_by_avg_coherence][:num_topics] # limit for HDP

    ordered_topic_averages = [topic_averages[i] for i in topic_indexes_by_avg_coherence][:num_topics] # limit for HDP
    ordered_topic_averages = [a/sum(ordered_topic_averages) for a in ordered_topic_averages] # normalize HDP values

    return ordered_topics, ordered_topic_averages

def getKeywordsLDA(group):

    #print((group[0]).split())
    """for x in range(len(group)):
        group[x] = group[x].split()"""
    dirichlet_dict = corpora.Dictionary(group)
    bow_corpus = [dirichlet_dict.doc2bow(text) for text in group]

    num_topics = 10
    num_keywords = 5

    dirichlet_model = LdaModel(corpus=bow_corpus,
                            id2word=dirichlet_dict,
                            num_topics=num_topics,
                            update_every=1,
                            chunksize=len(bow_corpus),
                            passes=20,
                            alpha='auto')

    ordered_topics, ordered_topic_averages = \
        order_subset_by_coherence(dirichlet_model=dirichlet_model,
                                bow_corpus=bow_corpus, 
                                num_topics=num_topics,
                                num_keywords=num_keywords)

    keywords = []
    for i in range(num_topics):
        # Find the number of indexes to select, which can later be extended if the word has already been selected
        selection_indexes = list(range(int(round(num_keywords * ordered_topic_averages[i]))))
        if selection_indexes == [] and len(keywords) < num_keywords: 
            # Fix potential rounding error by giving this topic one selection
            selection_indexes = [0]
                
        for s_i in selection_indexes:
            if ordered_topics[i][s_i] not in keywords:
                keywords.append(ordered_topics[i][s_i])
            else:
                selection_indexes.append(selection_indexes[-1] + 1)

    # Fix for if too many were selected
    keywords = keywords[:num_keywords]

    return keywords

def keywordResultsGroup(textGroups, func):
    textLabel = list(textGroups["Labels"])
    textArticles = list(textGroups["Text groups"])
    allKeywords = []
    for x in range(len(textLabel)):
        #print(textArticles[x])
        allKeywords.append(func(textArticles[x]))
    
    keywordDF = pd.DataFrame(data = {"Topic": textLabel, "Keywords": allKeywords})
    return keywordDF

print(keywordResultsGroup(textGroups, getKeywordsLDA).to_string())

In [ ]:
import yake

kw_extractor = yake.KeywordExtractor(top = 5)
keywords = kw_extractor.extract_keywords(text_l[15])
print(keywords)
print(text_l[1])
#print(keywords)
#print(getKeywordsYake(clusterLabels)[0])

In [ ]:
from summa import keywords

clusterLabelsNumpy = np.array(clusterLabels)
seen = []
groups = [""] * len(set(clusterLabels))
count = 0 
for x in range(len(clusterLabels)):
    if(clusterLabels[x] not in seen):
        seen.append(clusterLabels[x])
        locations = np.where(clusterLabelsNumpy == clusterLabels[x])[0].tolist()
        for x in (locations):
            groups[count] = groups[count] + " " + rawText[x]
        count = count + 1

keywordList = []
for x in groups:
    TR_keywords = keywords.keywords(x, scores=True)
    keywordList.append(TR_keywords[0:5])

print(keywordList)

In [103]:
from summa import keywords
TR_keywords = keywords.keywords(text_l[4], scores=True)
print(TR_keywords[0:5])

[('xmp', 0.3668069576946353), ('migrate', 0.3207280575858291), ('migrated', 0.3207280575858291), ('bird', 0.168915883542472), ('briedis etal', 0.16691740548376394)]


In [ ]:
from rake_nltk import Rake
r = Rake()
textList = """Google quietly rolled out a new way for Android users to listen 
to podcasts and subscribe to shows they like, and it already works on 
your phone. Podcast production company Pacific Content got the exclusive 
on it.This text is taken from Google news."""
r.extract_keywords_from_text(textList)
#r.get_ranked_phrases()[0:5]

In [ ]:
clusterLabelsNumpy = np.array(clusterLabels)
seen = []
groups = [""] * len(set(clusterLabels))
count = 0 
for x in range(len(clusterLabels)):
    if(clusterLabels[x] not in seen):
        seen.append(clusterLabels[x])
        locations = np.where(clusterLabelsNumpy == clusterLabels[x])[0].tolist()
        for x in (locations):
            groups[count] = groups[count] + " " + rawText[x]
        count = count + 1

keywordList = []
#kw_model = KeyBERT(model='all-mpnet-base-v2')
kw_model = KeyBERT(model = 'all-MiniLM-L6-v2')
for x in groups:
    currentKeyword = kw_model.extract_keywords(x, keyphrase_ngram_range=(1, 2), stop_words='english', highlight=False, top_n=5)
    keywordList.append(list(dict(currentKeyword).keys()))
    print(currentKeyword)
print(keywordList)

In [99]:
from keybert import KeyBERT
import itertools
from collections import Counter

df["Labels"] = labels
df["Text"] = text_l
textGroups = df.groupby("Labels")["Text"].apply(list).reset_index(name='Text groups')

#model="all-MiniLM-L6-v2"
#model='all-mpnet-base-v2'
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

textLabel = list(textGroups["Labels"])
textArticles = list(textGroups["Text groups"])
allKeywords = []

def getKeywordsKeyBERT(articles):
    total = []
    
    for x in articles:
        #print(x)
        keywordsTemp = kw_model.extract_keywords(docs = x, keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
        #print(keywordsTemp)
        total = total + list(itertools.chain.from_iterable(keywordsTemp))
    totalWords = total[0::2]
    #mostCommon = Counter(totalWords)
    #most_occur = mostCommon.most_common(3)
    #print(totalWords)
    most_common_words= [word for word, word_count in Counter(totalWords).most_common(3)]
    #print(totalWords)
    #print(most_common_words)
    return most_common_words

smallLabels = textLabel[0:2]
#print(smallLabels)
#print(len(smallLabels))
for x in range(len(textLabel)):
    print(x)
    allKeywords.append(getKeywordsKeyBERT(textArticles[x]))

keywordDF = pd.DataFrame(data = {"Topic": textLabel, "Keywords": allKeywords})

print(keywordDF.to_string())

"""keywords1 = kw_model.extract_keywords(docs = rawText[0], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
keywords1List = list(itertools.chain.from_iterable(keywords1))
keywords2 = kw_model.extract_keywords(docs = rawText[1], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
keywords2List = list(itertools.chain.from_iterable(keywords2))
keywords3 = kw_model.extract_keywords(docs = rawText[2], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
keywords3List = list(itertools.chain.from_iterable(keywords3))
total = keywords1List + keywords2List + keywords3List
totalWords = total[0::2]
mostCommon = Counter(totalWords)
most_occur = mostCommon.most_common(3)"""


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
                                      Topic                                 Keywords
0                                    Alloys           [metallurgy, aluminium, alloy]
1                           Avian Migration        [migratory, migrating, migration]
2                             Breash Cancer      [mammography, imaging, bcr_imaging]
3                                      Cake                 [baked, dessert, recipe]
4                           Circadian Rythm           [sleep, sleepiness, circadian]
5                               Court Cases           [judicial, judge, segregation]
6                            Food Allergies              [allergy, allergic, asthma]
7                                 Fugatives              [fugitive, suspect, arrest]
8                             Horror Movies       [cast, cinematography, screenplay]
9                    Humanized Mice Testing  [hiv, immunodeficiency, antiretroviral]
10  Intelligent Data Ana

'keywords1 = kw_model.extract_keywords(docs = rawText[0], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)\nkeywords1List = list(itertools.chain.from_iterable(keywords1))\nkeywords2 = kw_model.extract_keywords(docs = rawText[1], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)\nkeywords2List = list(itertools.chain.from_iterable(keywords2))\nkeywords3 = kw_model.extract_keywords(docs = rawText[2], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)\nkeywords3List = list(itertools.chain.from_iterable(keywords3))\ntotal = keywords1List + keywords2List + keywords3List\ntotalWords = total[0::2]\nmostCommon = Counter(totalWords)\nmost_occur = mostCommon.most_common(3)'

In [ ]:
print(textGroups.to_string())

In [81]:
from keybert import KeyBERT
import itertools
from collections import Counter

#model="all-MiniLM-L6-v2"
#model='all-mpnet-base-v2'
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

textLabel = list(textGroups["Labels"])
textArticles = list(textGroups["Text groups"])
allKeywords = []

"""def getKeywordsKeyBERT(articles):
    total = []
    for x in articles:
        keywordsTemp = kw_model.extract_keywords(docs = rawText[0], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
        total = total + keywordsTemp
    totalWords = total[0::2]
    mostCommon = Counter(totalWords)
    most_occur = mostCommon.most_common(3)
    return most_occur

smallLabels = textLabel[0]
print(len(smallLabels)

for x in range(len(smallLabels)):
    print(x)
    allKeywords.append(getKeywordsKeyBERT(textArticles[x]))

keywordDF = pd.DataFrame(data = {"Topic": textLabel[0], "Keywords": allKeywords})

print(keywordDF)"""

keywords1 = kw_model.extract_keywords(docs = rawText[0], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
keywords1List = list(itertools.chain.from_iterable(keywords1))
keywords2 = kw_model.extract_keywords(docs = rawText[1], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
keywords2List = list(itertools.chain.from_iterable(keywords2))
keywords3 = kw_model.extract_keywords(docs = rawText[2], keyphrase_ngram_range=(1, 1), highlight=False, top_n=10)
keywords3List = list(itertools.chain.from_iterable(keywords3))
total = keywords1List + keywords2List + keywords3List
totalWords = total[0::2]
mostCommon = Counter(totalWords)
most_occur = mostCommon.most_common(3)
most_common_words= [word for word, word_count in Counter(totalWords).most_common(3)]
print(total)
print(most_common_words)

['aluminium', 0.6596, 'aluminum', 0.6185, 'isotopes_of_aluminium', 0.6114, 'aluminium_alloy', 0.5896, 'aluminium_alloys', 0.5849, 'aluminium_oxide', 0.5802, 'aluminiumâ', 0.5726, 'alkali_metals', 0.5647, 'aluminium_arsenide', 0.564, 'native_aluminium', 0.5634, 'beryllium', 0.6361, 'beryllium_iron', 0.6233, 'berylliumâ', 0.6105, 'beryllium_copper', 0.599, 'beryllium_nitride', 0.5877, 'alkaline_earth_metal', 0.5729, 'alkali_metals', 0.5706, 'beryllium_oxide', 0.5686, 'beryllium_nitrate', 0.5664, 'beryllium_sulfate', 0.5657, 'isotopes_of_bismuth', 0.5462, 'bismuth_crystals', 0.5373, 'bismuth_oxychloride', 0.5115, 'bismuthyl', 0.5084, 'chemical_element', 0.4986, 'bismuthinite', 0.4859, 'bismoclite', 0.4843, 'bismuth_bronze', 0.4758, 'bismuth_oxide', 0.4705, 'bismuthine', 0.4696]
['alkali_metals', 'aluminium', 'aluminum']
